In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
y_val_original = pd.read_csv('y_val.csv')
y_val_original.head()

,is_female
0,1.0
1,0.0
2,0.0
3,1.0
4,1.0


In [4]:
neerja_cat_val = pd.read_csv('val_cat_11.csv')
neerja_xgb_val = pd.read_csv('val_xgb_11_non_mode.csv')
neerja_xgb_mode = pd.read_csv('val_xgb_11.csv')

In [5]:
alv_lgb_val = pd.read_csv('pred_val_lgb.csv', usecols=['predicted'])
alv_lgb_val.head()

,predicted
0,0.997593
1,0.035451
2,0.026996
3,0.998508
4,0.978705


In [6]:
alv_cat_val = pd.read_csv('pred_val_clf.csv', usecols=['predicted'])
alv_cat_val.head()

,predicted
0,0.996080
1,0.032184
2,0.015652
3,0.997759
4,0.993541


In [7]:
train = pd.concat([neerja_cat_val['0'], neerja_xgb_val['0'],neerja_xgb_mode['0'], alv_lgb_val['predicted'],alv_cat_val['predicted'], y_val_original['is_female']], axis = 1)
train.columns = ['neerja_cat', 'neerja_xgb','neerja_mode', 'alv_lgb', 'alv_cat', 'is_female']

In [8]:
train.head()

,neerja_cat,neerja_xgb,neerja_mode,alv_lgb,alv_cat,is_female
0,0.993414,0.998567,0.998604,0.997593,0.996080,1.0
1,0.035946,0.009263,0.006491,0.035451,0.032184,0.0
2,0.041512,0.012879,0.014130,0.026996,0.015652,0.0
3,0.994296,0.998016,0.998014,0.998508,0.997759,1.0
4,0.967507,0.978969,0.950808,0.978705,0.993541,1.0


In [9]:
y = train['is_female']
X = train.drop(['is_female'], axis = 1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=10)

In [10]:
X_train.shape, X_val.shape

((2920, 5), (731, 5))

In [11]:
X_train.head()

,neerja_cat,neerja_xgb,neerja_mode,alv_lgb,alv_cat
3561,0.076601,0.005657,0.005178,0.044871,0.033973
1144,0.028621,0.001201,0.001609,0.009748,0.013746
855,0.069472,0.006002,0.010632,0.012297,0.016707
565,0.965981,0.978285,0.984846,0.965408,0.960066
2705,0.379516,0.538000,0.430544,0.417673,0.393391


In [12]:
lr = LinearRegression(n_jobs=-1)
lr.fit(X_train, y_train)
pred_val = lr.predict(X_val)
lr.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [13]:
metrics.mean_squared_error(pred_val, y_val)

0.059398212057408868

In [14]:
neerja_cat_test = pd.read_csv('test_cat_11.csv')
neerja_xgb_test = pd.read_csv('test_xgb_11_full.csv')
neerja_xgb_mode_test = pd.read_csv('test_xgb_11_full_mode_imp.csv')
alv_lgb_test = pd.read_csv('submission_lgb_972.csv')
alv_cat_test = pd.read_csv('submissionclf_9715.csv')


In [15]:
test = pd.concat([neerja_cat_test['is_female'], neerja_xgb_test['is_female'], neerja_xgb_mode_test['is_female'],alv_lgb_test['is_female'], alv_cat_test['is_female']], axis = 1)
test.columns = ['neerja_cat', 'neerja_xgb','neerja_mode', 'alv_lgb', 'alv_cat']

In [16]:
test.head()

,neerja_cat,neerja_xgb,neerja_mode,alv_lgb,alv_cat
0,0.990503,0.999022,0.998421,0.997957,0.996031
1,0.055382,0.041221,0.026881,0.054901,0.090821
2,0.982427,0.973686,0.972797,0.985685,0.970349
3,0.963202,0.995936,0.988572,0.974083,0.971160
4,0.792713,0.539436,0.587616,0.623080,0.593792


In [18]:
lr.coef_

array([-0.27888408,  0.14652361,  0.13489476,  0.45124857,  0.54126803])

In [19]:
test_pred = lr.predict(test)

In [20]:
# lr.coef_ = np.array([-0.26615687, 0.48480678, 0.2388696 , 0.53854041])

In [21]:
test_pred

array([ 0.99935547,  0.0732361 ,  0.97500031, ...,  0.00991611,
        0.17294728,  0.81497751])

In [22]:
len(test_pred[test_pred > 1])

455

In [23]:
len(test_pred[test_pred < 0])

114

In [24]:
test_pred[test_pred > 1] = 1
test_pred[test_pred < 0] = 0

In [25]:
test_pred

array([ 0.99935547,  0.0732361 ,  0.97500031, ...,  0.00991611,
        0.17294728,  0.81497751])

In [26]:
sub = pd.read_csv('../sample_submission.csv')
sub.is_female = test_pred
sub.to_csv('stack3_5_models.csv', index = None)